REFERENCES

[1] R. Storn and K. Price, “Differential evolution – a simple and efficient heuristic for global optimization over continuous spaces,” Journal of Global Optimization, vol. 11, no. 4, pp. 341–359, 1997

[2] Lucas de P. Veronese and Renato A. Krohling, “Differential Evolution Algorithm on the GPU with C-CUDA,” Proc. of the IEEE Congress on Evolutionary Computation, Barcelona, Spain, Jul. 18-23, 2010, pp. 1-7.

Conventions: the index `j` addresses the population member while the index `i` addresses the member component the homologous host and device variables have the same name with a "`h_`" or "`d_`" prefix, respectively the `__host__` and `__device__` functions pointer parameters have the same name for comparison purposes. it is up to the caller to use	host or device pointers, as appropriate.


Rastrigrin functional: https://en.wikipedia.org/wiki/Rastrigin_function

https://en.wikipedia.org/wiki/Test_functions_for_optimization

In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scikit-cuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np

In [ ]:
import skcuda
from skcuda import cublas

/usr/local/lib/python3.7/dist-packages/skcuda/cublas.py:284: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [ ]:
import secrets

In [ ]:
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.compiler import SourceModule

In [ ]:
import pycuda.driver as drv
drv.init()
print("%d device(s) found." % drv.Device.count())
for i in range(drv.Device.count()):
  dev = drv.Device(i)
  print("Device #%d: %s" % (i, dev.name()))
  print(" Compute Capability: %d.%d" % dev.compute_capability())
  print(" Total Memory: %s GB" % (dev.total_memory() // (1024 * 1024 * 1024)))

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 14 GB


In [ ]:
###################
# iDivUp FUNCTION #
###################
def iDivUp(a, b):
    # Round a / b to nearest higher integer value
    a = np.int32(a)
    b = np.int32(b)
    return (a / b + 1) if (a % b != 0) else (a / b)

Constants.

In [ ]:
# --- De Jong function (hypersphere)
#MINIMA                = -5.12
#MAXIMA                =  5.12
# --- Rosenbrock's saddle - xopt = (1., 1., ..., 1.)
#MINIMA                = -2.048
#MAXIMA                =  2.048
# --- Rastrigin - xopt = (0., 0., ..., 0.) - fopt = 0.
#MINIMA                = -5.12
#MAXIMA                =  5.12
# --- Schwfel - xopt(-420.9698, -420.9698, ..., -420.9698)
MINIMA                  = -512.03
MAXIMA                  =  511.97

BLOCK_SIZE_POP	        = 32
BLOCK_POP               = 256
BLOCK_SIZE_RAND1	      = 64
BLOCK_SIZE_RAND2	      = 64
BLOCK_SIZE_UNKN		      = 8
BLOCK_SIZE			        = 256

Parameters.

In [ ]:
# --- Number of population members (Np >=4 for mutation purposes)
Np        = 800
# --- Problem dimensionality (number of unknowns)
D         = 5
# --- Mutation factor (0 < F <= 2). Typically chosen in [0.5, 1], see Ref. [1]
F         = 0.7
# --- Maximum number of generations (iterations)
Gmax      = 18000
# --- Crossover constant (0 < CR <= 1)
CR        = 0.4

Host-side allocations.

In [ ]:
# --- Host side population best value history of device side
h_best_dev        = np.empty((Gmax, 1), np.float32)				

# --- Host side population history index corresponding to the best value of device side
h_best_index_dev  = np.empty((Gmax, 1), np.int32)			

# --- Host side maximum constraints vector
h_maxima          = np.empty((D, 1), np.float32)

# --- Host side minimum constraints vector
h_minima          = np.empty((D, 1), np.float32)

# --- Set maxima and minima
for i in range(D):
  h_maxima[i] = MAXIMA
  h_minima[i] = MINIMA

Device-side allocations.

In [ ]:
# --- Mutually different random integer indices selected from {1, 2, … ,Np}
# --- Device side mutation vector
d_mutation    = gpuarray.empty((Np, 3), np.int32)    

# --- Device side population
d_pop         = gpuarray.empty((Np, D), np.float32) 

# --- Device side new population (trial vectors)
d_npop        = gpuarray.empty((Np, D), np.float32) 

# --- Device side crossover rand vector (uniformly distributed in (0, 1))
d_Rand        = gpuarray.empty((Np, D), np.float32) 

# --- Device side objective function value
d_fobj        = gpuarray.empty((Np, 1), np.float32) 
	
# --- Device side maximum constraints vector
d_maxima      = gpuarray.to_gpu(h_maxima)

# --- Device side minimum constraints vector
d_minima      = gpuarray.to_gpu(h_minima)

# --- Device side crossover rand vector (uniformly distributed in (0,1))
d_Rand        = gpuarray.empty((Np, 1), np.float32) 

Kernels.

In [ ]:
mod = SourceModule("""
   
#include <curand_kernel.h>

const int nstates = %(NGENERATORS)s;
__device__ curandState_t* states[nstates];

#define PI_f				3.14159265358979f

/****************************************/
/* EVALUATION OF THE OBJECTIVE FUNCTION */
/****************************************/
__global__ void curand_setup_kernel(const unsigned long int seed) {

	int tidx = threadIdx.x + blockIdx.x * blockDim.x;

  if (tidx < nstates) {
		curandState_t* s = new curandState_t;
		if (s != 0) curand_init(seed, tidx, 0, s);
		states[tidx] = s;
}}

/********************************/
/* INITIALIZE POPULATION ON GPU */
/********************************/
__global__ void initialize_population_GPU(float * __restrict pop, const float * __restrict minima, const float * __restrict maxima, const int D, const int Np) {

	int i = threadIdx.x + blockIdx.x * blockDim.x;
	int j = threadIdx.y + blockIdx.y * blockDim.y;

	if ((i < D) && (j < Np)) {
		curandState_t s = *states[j * D + i];
		pop[j * D + i] = (maxima[i] - minima[i]) * curand_uniform(states[j * D + i]) + minima[i];
		*states[j * D + i] = s;
}}

__host__ __device__ float functional(const float * __restrict x, const int D) {

	// --- More functionals at https://www.cs.cmu.edu/afs/cs/project/jair/pub/volume24/ortizboyer05a-html/node6.html

	float sum;
	// --- De Jong function (hypersphere)
	//sum = 0.f;
	//for (int i=0; i<D; i++) sum = sum + x[i] * x[i];
	// --- Rosenbrock's saddle - xopt = (1., 1., ..., 1.)
	//sum = 0.f;
	//for (int i = 1; i<D; i++) sum = sum + 100.f * (x[i] - x[i - 1] * x[i - 1]) * (x[i] - x[i - 1] * x[i - 1]) + (x[i - 1] - 1.f) * (x[i - 1] - 1.f);
	// --- Rastrigin - xopt = (0., 0., ..., 0.)
	//sum = 10.f * D;
	//for (int i = 1; i <= D; i++) sum = sum + (x[i - 1] * x[i - 1] - 10.f * cos(2.f * PI_f * x[i - 1]));
	// --- Schwfel - xopt(-420.9698, -420.9698, ..., -420.9698)
	sum = 418.9829 * D;
	for (int i = 1; i <= D; i++) sum = sum + x[i - 1] * sin(sqrt(fabs(x[i - 1])));

	return sum;
}

/********************************/
/* POPULATION EVALUATION ON GPU */
/********************************/
__global__ void evaluation_GPU(const int Np, const int D, const float * __restrict pop, float * __restrict fobj) {

	int j = threadIdx.x + blockIdx.x * blockDim.x;

	if (j < Np)  fobj[j] = functional(&pop[j * D], D);
}

/**********************************************************/
/* GENERATE MUTATION INDICES AND CROSS-OVER VALUES ON GPU */
/**********************************************************/
__global__ void generate_mutation_indices_GPU(int * __restrict mutation, const int Np, const int D) {

	int j = threadIdx.x + blockIdx.x * blockDim.x;

	int a, b, c;

	if (j < Np) {

		//curandState_t s = *states[j * D];
		curandState_t s = *states[j];
		do a = Np * (curand_uniform(states[j]));	while (a == j);
		do b = Np * (curand_uniform(states[j]));	while (b == j || b == a);
		do c = Np * (curand_uniform(states[j]));	while (c == j || c == a || c == b);
		mutation[j * 3] = a;
		mutation[j * 3 + 1] = b;
		mutation[j * 3 + 2] = c;

}}

/**********************************************************/
/* GENERATE MUTATION INDICES AND CROSS-OVER VALUES ON GPU */
/**********************************************************/
__global__ void generate_crossover_values_GPU(float * __restrict Rand, const int Np, const int D) {

	int j = threadIdx.x + blockIdx.x * blockDim.x;

	curandState_t s = *states[j];
	
	if (j < Np) Rand[j] = curand_uniform(states[j]);
}

/**********************************/
/* GENERATION OF A NEW POPULATION */
/**********************************/
__global__ void generation_new_population_GPU(const float * __restrict pop, const int Np, const int D, float * __restrict npop, const float F,
	const float CR, const float * __restrict rand, const int * __restrict mutation,
	const float * __restrict minimum, const float * __restrict maximum) {

	int i = threadIdx.x + blockIdx.x * blockDim.x;
	int j = threadIdx.y + blockIdx.y * blockDim.y;

	if ((i < D) && (j < Np)) {

		// --- Mutation indices
		int a = mutation[j * 3];
		int b = mutation[j * 3 + 1];
		int c = mutation[j * 3 + 2];

		// --- Mutation and binomial crossover
		// --- DE/rand/1. One of the best strategies. Try F = 0.7 and CR = 0.5 as a first guess.
		//if (rand[j*D + i]<CR)	npop[j*D + i] = pop[a*D + i] + F*(pop[b*D + i] - pop[c*D + i]);
		if (rand[j]<CR)	npop[j*D + i] = pop[a*D + i] + F*(pop[b*D + i] - pop[c*D + i]);
		else			npop[j*D + i] = pop[j*D + i];

		// --- Other possible approaches to mutation and crossover
		// --- DE/best/1 --- Not bad, but found several optimization problems where misconvergence occurs.
		//npop[j*D+i] = pop[best_old_gen_ind*D+i] + F*(pop[b*D+i]-pop[c*D+i]);
		// --- DE/rand to best/1 --- F1 can be different or equal to F2
		//npop[j*D+i] = pop[a*D+i] + F1*(pop[best_old_gen_ind*D+i] - pop[a*D+i]) + F2*(pop[b*D+i]-pop[c*D+i]);
		// --- DE/current to best/1 --- One of the best strategies. Try F = 0.85 and CR = 1. In case of misconvergence, try to increase Np. If this doesn't help,
		//     play around with all the control variables --- F1 can be different or equal to F2
		//npop[j*D+i] = pop[j*D+i] + F1*(pop[best_old_gen_ind*D+i] - pop[j*D+i]) + F2*(pop[a*D+i]-pop[b*D+i]);
		// --- DE/rand/2 --- Robust optimizer for many functions.
		//npop[j*D+i] = pop[e*D+i] + F*(pop[a*D+i]+pop[b*D+i]-pop[c*D+i]-pop[d*D+i]);
		// --- DE/best/2 --- Powerful strategy worth trying.
		//npop[j*D+i] = pop[best_old_gen_ind*D+i] + F*(pop[a*D+i]+pop[b*D+i]-pop[c*D+i]-pop[d*D+i]);

		// --- Saturation due to constraints on the unknown parameters
		if (npop[j*D + i]>maximum[i])		npop[j*D + i] = maximum[i];
		else if (npop[j*D + i]<minimum[i])	npop[j*D + i] = minimum[i];
}}

/*******************************/
/* POPULATION SELECTION ON GPU */
/*******************************/
// Assumption: all the optimization variables are associated to the same thread block
__global__ void selection_and_evaluation_GPU(const int Np, const int D, float * __restrict pop, const float * __restrict npop, float * __restrict fobj) {

	int i = threadIdx.x;
	int j = threadIdx.y + blockIdx.y * blockDim.y;

	if ((i < D) && (j < Np)) {

		float nfobj = functional(&npop[j*D], D);

		float temp = fobj[j];

		if (nfobj < temp) {
			pop[j*D + i] = npop[j*D + i];
			fobj[j] = nfobj;
}}}
  
  """ % { "NGENERATORS" : Np * D }, no_extern_c = True, arch = "sm_75")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:163: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(80): warning: variable "s" was declared but never referenced

kernel.cu(97): warning: variable "s" was declared but never referenced




Name mangling: nvcc test.cu -ptx

In [ ]:
curand_setup_kernel             = mod.get_function("_Z19curand_setup_kernelm")
initialize_population_GPU       = mod.get_function("_Z25initialize_population_GPUPfPKfS1_ii")
evaluation_GPU                  = mod.get_function("_Z14evaluation_GPUiiPKfPf")
generate_mutation_indices_GPU   = mod.get_function("_Z29generate_mutation_indices_GPUPiii")
generate_crossover_values_GPU   = mod.get_function("_Z29generate_crossover_values_GPUPfii")
generation_new_population_GPU   = mod.get_function("_Z29generation_new_population_GPUPKfiiPfffS0_PKiS0_S0_")
selection_and_evaluation_GPU    = mod.get_function("_Z28selection_and_evaluation_GPUiiPfPKfS_")

In [ ]:
GridEvaluation        = (int(iDivUp(Np, BLOCK_POP)), 1, 1)
BlockEvaluation       = (BLOCK_POP, 1, 1)

GridcuRandSetup       = (int(iDivUp(D * Np, BLOCK_SIZE)), 1, 1)
BlockcuRandSetup      = (BLOCK_SIZE, 1, 1)

GridMutationIndices   = (int(iDivUp(Np, BLOCK_SIZE_RAND1)), 1, 1)
BlockMutationIndices  = (BLOCK_SIZE_RAND1, 1, 1)

GridCrossover         = (int(iDivUp(Np, BLOCK_SIZE_RAND2)), 1, 1)
BlockCrossover        = (BLOCK_SIZE_RAND2, 1, 1)

Grid                  = (int(iDivUp(D, BLOCK_SIZE_UNKN)), int(iDivUp(Np, BLOCK_SIZE_POP)), 1)
Block                 = (BLOCK_SIZE_UNKN, BLOCK_SIZE_POP, 1)

In [ ]:
# --- cuBLAS context creation
h = cublas.cublasCreate()

In [ ]:
# --- Initialize cuRAND states
curand_setup_kernel(np.uint(secrets.randbelow(1_000_000_000)), block = BlockcuRandSetup, grid = GridcuRandSetup)

In [ ]:
# --- Initialize popultion
initialize_population_GPU(d_pop, d_minima, d_maxima, np.int32(D), np.int32(Np), block = Block, grid = Grid)

In [ ]:
for i in range(Gmax):
  # --- Evaluate population
  evaluation_GPU(np.int32(Np), np.int32(D), d_pop, d_fobj, block = BlockEvaluation, grid = GridEvaluation)

  # --- Generate mutation indices
  generate_mutation_indices_GPU(d_mutation, np.int32(Np), np.int32(D), block = BlockMutationIndices, grid = GridMutationIndices)

  # --- Generate crossover values
  generate_crossover_values_GPU(d_Rand, np.int32(Np), np.int32(D), block = BlockCrossover, grid = GridCrossover)

  # --- Generate new population
  #print(d_minima)
  generation_new_population_GPU(d_pop, np.int32(Np), np.int32(D), d_npop, np.float32(F), np.float32(CR), d_Rand, d_mutation, d_minima, d_maxima, block = Block, grid = Grid)

  # --- Select new population and evaluate it
  selection_and_evaluation_GPU(np.int32(Np), np.int32(D), d_pop, d_npop, d_fobj, block = Block, grid = Grid)

  m = cublas.cublasIsamin(h, d_fobj.size, d_fobj.gpudata, 1)

  h_best_index_dev[i] = m

  h_best_dev[i] = d_fobj[m].get()

  print("Iteration: {}; best member value: {}: best member index: {}\n".format(i, h_best_dev[i], h_best_index_dev[i]))

Output streaming troncato alle ultime 5000 righe.
Iteration: 15500; best member value: [230.08809]: best member index: [0]

Iteration: 15501; best member value: [230.08809]: best member index: [0]

Iteration: 15502; best member value: [230.08809]: best member index: [0]

Iteration: 15503; best member value: [230.08809]: best member index: [0]

Iteration: 15504; best member value: [230.08809]: best member index: [0]

Iteration: 15505; best member value: [230.08809]: best member index: [0]

Iteration: 15506; best member value: [230.08809]: best member index: [0]

Iteration: 15507; best member value: [230.08809]: best member index: [0]

Iteration: 15508; best member value: [230.08809]: best member index: [0]

Iteration: 15509; best member value: [230.08809]: best member index: [0]

Iteration: 15510; best member value: [230.08809]: best member index: [0]

Iteration: 15511; best member value: [230.08809]: best member index: [0]

Iteration: 15512; best member value: [230.08809]: best member 

In [ ]:
# --- Host side population result of GPU computations
h_pop_dev_res = d_pop.get()

for i in range(D):
  print("Variable nr. {} = {}\n".format(i, h_pop_dev_res[h_best_index_dev[Gmax - 1] * D , i]))

Variable nr. 0 = [-420.969]

Variable nr. 1 = [511.97]

Variable nr. 2 = [-420.97995]

Variable nr. 3 = [-420.97116]

Variable nr. 4 = [511.97]



In [ ]:
cublas.cublasDestroy(h)